In [ ]:
"""
All imports
"""

import zoopla # Python wrapper for Zoopla API. Installed through pip.
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.6f' % x,'display.max_columns', None, 'display.max_rows', None)

# To get latest file in directory
import glob
import os
import datetime
from random import randrange

# For simple linear regression on area values
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression

In [ ]:
"""
Regression on zed indices to find areas of increasing value. Also adds whether auction or not
"""

list_of_files = glob.glob('.../scraper/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file,index_col=0)
# df = df.transpose()

# Adding columns
df[['5_year_reg','4_year_reg','3_year_reg','2_year_reg','1_year_reg','6_month_reg','3_month_reg']] = df[['zed_index_5year','zed_index_4year','zed_index_3year','zed_index_2year','zed_index_1year','zed_index_6month','zed_index_3month']].apply(pd.to_numeric)
df['average_sale_price-price'] = df['sale_price'].apply(pd.to_numeric)
df['price']=df['price'].apply(pd.to_numeric)
df['average_sale_price-price'] = df['sale_price'].sub(df['price'], axis=0 )

c = 0
drop = 0

for i, row in df.iterrows():
    c += 1
    #print "---- NEW -----"
    print c
    if float(row['zed_index']) == 0.0:
        df.drop(i,inplace=True)
        print "DROP"
        drop += 1
    else:
        if "auction" in row['description']:
            df.loc[i,'auction'] = 1
        Y = np.array([float(row['zed_index_5year']),float(row['zed_index_4year']),float(row['zed_index_3year']),float(row['zed_index_2year']),float(row['zed_index_1year']),float(row['zed_index_6month']),float(row['zed_index_3month']),float(row['zed_index'])])
        X = np.array([0,1,2,3,4,5,5.5,5.75])
        X = np.reshape(X,(8,1))
        Y = np.reshape(Y,(8,1))
        col_names = ['5_year_reg','4_year_reg','3_year_reg','2_year_reg','1_year_reg','6_month_reg','3_month_reg']
        col_names_counter = 0
        model = LinearRegression()
        model.fit(X,Y)
#         if float(row['zed_index']) == 0.0:
#             continue
        df.loc[i,col_names[col_names_counter]] = float(model.coef_[0])/float(row['zed_index'])
        col_names_counter += 1
        for x in range(len(X)- 2):
            X = np.delete(X, 0)
            Y = np.delete(Y, 0)
            X = np.reshape(X,(8-(x+1),1))
            Y = np.reshape(Y,(8-(x+1),1))
            model = LinearRegression()
            model.fit(X,Y)
            df.loc[i,col_names[col_names_counter]] = float(model.coef_[0])/float(row['zed_index'])
            col_names_counter += 1
        df.loc[i,'1_year_reg_rate'] = (row['1_year_reg'] - row['2_year_reg'])
        df.loc[i,'6_month_reg_rate'] = (row['6_month_reg'] - row['1_year_reg'])/2. # Divided by two to regularise for time period of 6 months
        df.loc[i,'3_month_reg_rate'] = (row['3_month_reg'] - row['6_month_reg'])/4. # Divided by four to regularise for time period of 3 months

# Ordering
df = df[['displayable_address','details_url','distance_work_2','distance_work_1','cycling_work_2','agent_name','agent_phone','county','description','first_published_date','last_published_date','latitude','longitude','listing_status','listing_id','num_bathrooms','num_bedrooms','num_recepts','outcode','post_town','property_type','price_put_on_market','page_views_30_days','page_views_all_time','overall_rating','community_and_safety','entertainment_and_nightlife','parks_and_recreation','restaurants_and_shopping','schools_and_public_services','transport_and_travel', 'price', 'rental_price','sale_price', 'average_sale_price-price' ,'zed_index','zed_index_3month','zed_index_6month','zed_index_1year','zed_index_2year','zed_index_3year','zed_index_4year','zed_index_5year','5_year_reg','4_year_reg','3_year_reg','2_year_reg','1_year_reg','6_month_reg','3_month_reg','1_year_reg_rate','6_month_reg_rate','3_month_reg_rate','auction']]
print "Dropped: ",drop
now = datetime.datetime.now().strftime("%H%M%d%m%y")
df.to_csv(".../final/"+str(now)+".csv",encoding='utf-8')